# Code for image processing post segmentation

This notebook is made to select images with various channel intensities, crop and segment the nuclei to train an object classifier in ilastik.

In [80]:
import os 
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.io import imshow
from skimage.io import imread as skimage_imread
from tifffile import imread, imwrite
from skimage.measure import regionprops_table
from tqdm import tqdm
import random
import plotly.express as px
import plotly.graph_objects as go

from stardist import export_imagej_rois
from stardist.models import StarDist2D
from zipfile import ZIP_DEFLATED
from csbdeep.utils import normalize

In [ ]:
pwd

In [ ]:
# Define the different path of the folder containing your images to quantify
dir_img = '../img_test_pipeline'
dir_mask = dir_img + '/lbl'
dir_csv = dir_img

In [85]:
# Get a list of all the files contained in the directories:
tif_img = [f for f in os.listdir(dir_img) if f.endswith((".tif", ".tiff")) and not f.startswith("._")]
tif_mask = [f for f in os.listdir(dir_mask) if f.endswith((".tif", ".tiff")) and not f.startswith("._")]

# print the number of images in each folder:
total_img = len(tif_img)
total_mask = len(tif_mask)
print(f' You have {total_img} images and {total_mask} masks')

# Verify that the number of images and masks are identical
if len(tif_img) != len(tif_mask):
    raise ValueError("Number of images and number of masks images do not correspond")

 You have 12 images and 12 masks


In [86]:
# Extract the data from one random file to see the correpsonding matching
tif_files = [f for f in os.listdir(dir_img) if f.endswith(('.tif', '.tiff')) and not f.startswith("._")]
rfilename = random.choice(tif_files)

# Data extraction from the filename: this is specific to our usecase
parts = rfilename.split('_')
clone, dif, day, immuno, obj, imaging, og, s = parts[0], parts[1], parts[2], parts[3], parts[4], parts[5], parts[6], parts[7].split('.')[0]
print(f"clone: {clone}, dif: {dif}, day: {day}, immuno: {immuno}, obj: {obj}, imaging: {imaging}, og: {og}, s: {s}")

clone: Pa, dif: dQ, day: J70, immuno: SNd2T, obj: 20Xa, imaging: g1dt08, og: OG3, s: s2


In [ ]:
# Create an empty list for storing the data 
df_list = []

# Loop through the TIFF files and display progress: the enumerate function also use the index to match the img with its mask
with tqdm(total=total_img, ncols=80) as pbar:
    for index, image_file in enumerate(tif_img, 1):

        # Check if the corresponding label file exists
        label_file = image_file.replace('.tif', '_labels.tif')
        if label_file not in tif_mask:
            raise ValueError(f"Label file {label_file} not found for image {image_file}")

        # Load the image and label
        image = skimage_imread(os.path.join(dir_img, image_file))
        label = skimage_imread(os.path.join(dir_mask, label_file))

        if image.shape[:2] != label.shape:
            raise ValueError(f"Label image {label_file} and your image {image_file} don't have the same shape")

        # Assuming you have a function to extract the measurements using regionprops_table
        properties = regionprops_table(label, intensity_image=image, properties=('label', 'area', 'mean_intensity'))
        
        # Data extraction from filename:
        parts = image_file.split('_')
        clone, dif, day, immuno, obj, imaging, og, s = parts[0], parts[1], parts[2], parts[3], parts[4], parts[5], parts[6], parts[7].split('.')[0]

        # Add additional information to the properties dictionary
        properties['Image_name'] = image_file
        properties['Clone'] = clone
        properties['Diff'] = dif
        properties['Day'] = day
        properties['Immuno'] = immuno
        properties['Objective'] = obj
        properties['Imaging'] = imaging 
        properties['Organoid'] = og
        properties['Slice'] = s

        # Append to the list
        df_list.append(pd.DataFrame(properties))

        # Update the progress bar
        pbar.update(1)

# Concatenate all the DataFrames into a single DataFrame
result_df = pd.concat(df_list, ignore_index=True)
result_df = result_df.reset_index(drop=True)
df = result_df[['Image_name', 'Clone', 'Diff', 'Day', 'Immuno', 'Objective', 'Imaging', 'Organoid', 'Slice', 'label', 'area', 'mean_intensity-0', 'mean_intensity-1', 'mean_intensity-2', 'mean_intensity-3']]


# end
print('\n Process finished')


100%|███████████████████████████████████████████| 12/12 [00:51<00:00,  4.29s/it]



 Process finished


In [88]:
df.head()

,Image_name,Clone,Diff,Day,Immuno,Objective,Imaging,Organoid,Slice,label,area,mean_intensity-0,mean_intensity-1,mean_intensity-2,mean_intensity-3
0,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s1.tif,Pa,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s1,1,38,1119.263158,138.500000,126.552632,72.131579
1,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s1.tif,Pa,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s1,2,41,1018.024390,126.390244,119.390244,60.219512
2,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s1.tif,Pa,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s1,3,32,1366.500000,107.156250,118.687500,65.750000
3,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s1.tif,Pa,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s1,4,36,1149.333333,111.333333,109.527778,65.916667
4,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s1.tif,Pa,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s1,5,37,1321.297297,113.081081,126.189189,64.027027


In [89]:
# Dictionary of the clones and their corresponding genotypes: CHANGE IT ACCORDINGLY TO YOURS!!
genotypes = {
    'Pa':'WT/WT',
    'B6WT': 'WT/WT',
    'H6':'KO/KO',
    'C6':'KO/KO'
}

In [90]:
# Add a column to the df to add the appropriate genotype to each clone

In [91]:
# --- Step 2: Calculate mean intensities for ALL channels in one go ---
mean_intensities = result_df.groupby(['Image_name', 'Clone']).agg({
    'mean_intensity-0': 'mean',
    'mean_intensity-1': 'mean',
    'mean_intensity-2': 'mean',
    'mean_intensity-3': 'mean'
}).reset_index()

# Rename columns for clarity (optional)
mean_intensities = mean_intensities.rename(columns={
    'mean_intensity-0': 'Channel_1',
    'mean_intensity-1': 'Channel_2',
    'mean_intensity-2': 'Channel_3',
    'mean_intensity-3': 'Channel_4'
})

In [92]:
mean_intensities.head()

,Image_name,Clone,Channel_1,Channel_2,Channel_3,Channel_4
0,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s1.tif,Pa,665.377406,149.423085,123.425620,74.661118
1,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s2.tif,Pa,1003.023859,161.784087,126.813420,74.975994
2,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s3.tif,Pa,1196.752403,206.428304,173.055226,77.425020
3,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s4.tif,Pa,1211.058684,199.837467,150.183514,80.148175
4,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG2_s1.tif,Pa,1169.951995,202.096217,178.006865,85.845991


In [93]:
# Define here your channel of interest to perform all the following plotting and measurments 
y = 'Channel_2' # to be modified by the user

In [94]:
# --- Step 3: Plot any channel by just changing the column name ---
fig = px.scatter(
    mean_intensities,
    x='Clone',
    y=y,  # <-- Only this column name changes per channel
    color='Image_name',
    symbol='Image_name',
    opacity=0.7,
    title=f"Mean Intensity {y} by Clone"
)

# Overlay boxplots
box_trace = go.Box(
    x=mean_intensities['Clone'],
    y=mean_intensities[y],  # <-- Same column name here
    boxpoints='outliers',
    jitter=0.3,
    pointpos=-1.8,
    boxmean='sd',
    name='Distribution'
)
fig.add_trace(box_trace)

In [95]:
# Define w
output_base = os.path.join(dir_img, "crops")  # Dossier de sortie

# Create the subfolder to store the images and their corresponding masks
os.makedirs(os.path.join(output_base, "images"), exist_ok=True)
os.makedirs(os.path.join(output_base, "masks"), exist_ok=True)

# Calcuale the quantiles to select images with low, medium and high channel intensities:
quantiles = (
    mean_intensities
    .groupby('Clone')[y]
    .quantile([0.10, 0.50, 0.90])
    .unstack()
    .rename(columns={0.10: 'Q10', 0.50: 'Q50', 0.90: 'Q90'})
)

# Select the closest images to Q10, Q50 and Q90:
selected_img = []
for clone, q_values in quantiles.iterrows():
    clone_data = mean_intensities[mean_intensities['Clone'] == clone]
    for q in ['Q10', 'Q50', 'Q90']:
        idx = (clone_data[y] - q_values[q]).abs().idxmin()
        selected_img.append(clone_data.loc[idx])

# Create the df with the selected images
selected_img_df = pd.DataFrame(selected_img)

# Define the function to crop and save the selected images and their corresponding masks
def crop_pair(image_path, mask_path, output_img_path, output_mask_path, crop_factor_y=2, crop_factor_x=2):
    try:
        # read the images
        img = imread(image_path)  # Shape: (C, Y, X)
        mask = imread(mask_path)  # Shape: (Y, X)

        # Dimensions
        c, h, w = img.shape  # C, Y, X
        h_mask, w_mask = mask.shape  # Y, X

        # Vérifier que les dimensions correspondent
        if h != h_mask or w != w_mask:
            raise ValueError(f"Dimensions are not identical : image {img.shape} vs mask {mask.shape}")

        # Crop of the bottom left 
        # Y: de h//2 à h (low)
        # X: de 0 à w//2 (left)
        img_cropped = img[:, h//crop_factor_y:, :w//crop_factor_x]  # (C, Y/2, X/2)
        mask_cropped = mask[h//crop_factor_y:, :w//crop_factor_x]   # (Y/2, X/2)

        # Save
        imwrite(output_img_path, img_cropped)
        imwrite(output_mask_path, mask_cropped)

    except Exception as e:
        print(f"Error with {os.path.basename(image_path)}: {e}")


# Use the function
for _, row in selected_img_df.iterrows():
    # Paths to files
    img_name = row['Image_name']
    img_path = os.path.join(dir_img, img_name)
    mask_path = os.path.join(dir_mask, img_name.replace('.tif', '_labels.tif'))

    # Output paths
    output_img_path = os.path.join(output_base, "images", img_name)
    output_mask_path = os.path.join(output_base, "masks", img_name.replace('.tif', '_mask.tif'))

    # Crop and save
    crop_pair(img_path, mask_path, output_img_path, output_mask_path)

print(f"Crops done. Stored in : {output_base}/")


Crops done. Stored in : ../test_pipeline/crops/


In [97]:
# Save the df you created with all the raw data. It should take in between 30 to 60 minutes if you have many images
df.to_csv(f'../data/Raw_data_{dif}_{day}_{immuno}.csv', index=False)